# Лекция 1: Въведение в Изкуствен Интелект и Машинно Обучение

## Фокус: Обработка на Естествен Език (NLP)

**Продължителност:** 2-2.5 часа  
**Следваща лекция:** Езикови модели и представяне на думи

---
## 1. Мотивация: Защо AI и ML за език?

### Какво е AI, ML и NLP?

**🤖 Изкуствен Интелект (AI)**  
Широка област, която се занимава със създаване на интелигентни системи, способни да решават задачи, които изискват човешка интелигентност.

**📊 Машинно Обучение (ML)**  
Подобласт на AI, където системите **учат от данни**, вместо да бъдат програмирани експлицитно.

**💬 Обработка на Естествен Език (NLP)**  
Приложение на ML за разбиране, интерпретиране и генериране на човешки език.

### Защо езикът е труден за компютрите?

1. **Многозначност**: "Банката е затворена" (финансова институция или речен бряг?)
2. **Контекст**: "Чувствам се син" срещу "Небето е синьо"
3. **Сарказъм и ирония**: "Страхотно, пак закъснях..." 😒
4. **Неясности**: "Видях човек с телескоп" (кой има телескопа?)

### Успешни приложения на NLP

✅ **Машинен превод**: Google Translate, DeepL  
✅ **Чатботове**: ChatGPT, Claude, Siri  
✅ **Анализ на настроения**: Мониторинг на социални медии  
✅ **Обобщаване на текст**: Генериране на резюмета на статии  
✅ **Разпознаване на наименовани обекти**: Извличане на имена, места, организации

---
## 2. Формулиране на ML задача

### Формална дефиниция

В машинното обучение искаме да научим функция $h: \mathcal{X} \to \mathcal{Y}$, която преобразува входове в изходи.

**Ключови понятия:**
- $\mathcal{X}$: **Входно пространство** (например: текстови документи, изречения, думи)
- $\mathcal{Y}$: **Изходно пространство** (например: етикети, преводи, настроения)
- $(x^{(i)}, y^{(i)})$: **Тренировъчни примери** (входно-изходни двойки)
- $h$: **Хипотеза/Модел** (функцията, която учим)

### Конкретен NLP пример: Класификация на спам имейли

**Задача**: Определи дали имейл е спам или не.

- **Вход** ($x$): Текстът на имейла
- **Изход** ($y$): {"спам", "не е спам"}
- **Обучение**: Използваме хиляди етикетирани имейли
- **Цел**: Моделът да може да класифицира нови, невиждани имейли

**Какво означава "учене" тук?**  
Моделът открива модели в текста (думи като "спечели", "безплатно", много удивителни) които корелират със спам.

---
## 3. Основна терминология с текстови данни

### Характеристики (Features) в NLP

За да обработваме текст, трябва да го представим като числа. Ето няколко начина:

#### 1. Чанта с думи (Bag of Words)
Броим колко пъти се появява всяка дума в документа, игнорирайки реда.

**Пример:**
- Изречение: "Харесвам този филм. Този филм е страхотен!"
- Характеристики: {"харесвам": 1, "този": 2, "филм": 2, "е": 1, "страхотен": 1}

#### 2. TF-IDF (Term Frequency - Inverse Document Frequency)
По-умна схема за претегляне: често срещани думи в документа, но редки в корпуса, получават висока стойност.

#### 3. N-грами
Последователности от N думи:
- Уни-грами: ["харесвам", "този", "филм"]
- Би-грами: ["харесвам този", "този филм"]
- Три-грами: ["харесвам този филм", "този филм е"]

### Етикети (Labels)
- **Класификация на настроения**: {положително, отрицателно, неутрално}
- **Разпознаване на наименовани обекти**: {лице, организация, местоположение, ...}
- **Следваща дума**: В езиковите модели предсказваме следващата дума

### Разделяне на данните
- **Тренировъчен набор (70-80%)**: Моделът учи от тези данни
- **Валидационен набор (10-15%)**: Настройваме хиперпараметри
- **Тестов набор (10-15%)**: Финална оценка на модела

### Нека да видим това в действие!

In [ ]:
# Импортиране на необходимите библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Настройка на визуализациите
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Всички библиотеки са заредени успешно!")

### Демонстрация: Bag of Words и TF-IDF

In [ ]:
# Примерни документи на български
documents = [
    "Харесвам този филм. Този филм е страхотен!",
    "Не харесвам този филм. Филмът е ужасен.",
    "Този филм е добър. Препоръчвам го.",
    "Филмът беше скучен и дълъг."
]

print("📄 Нашите документи:")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc}")

# Bag of Words
print("\n" + "="*60)
print("🎒 BAG OF WORDS представяне")
print("="*60)

vectorizer_bow = CountVectorizer()
bow_matrix = vectorizer_bow.fit_transform(documents)

# Показваме речника
vocab = vectorizer_bow.get_feature_names_out()
print(f"\nРечник ({len(vocab)} уникални думи):")
print(vocab)

# Показваме матрицата
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vocab)
print("\nМатрица документ-термин (Bag of Words):")
print(bow_df)

# TF-IDF
print("\n" + "="*60)
print("📊 TF-IDF представяне")
print("="*60)

vectorizer_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizer_tfidf.fit_transform(documents)

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vocab)
print("\nМатрица документ-термин (TF-IDF):")
print(tfidf_df.round(3))

print("\n💡 Забележка: В TF-IDF често срещаните думи (като 'филм') получават по-ниски тегла,")
print("   докато по-рядко срещаните думи (като 'препоръчвам') получават по-високи тегла.")

---
## 4. Парадигми на обучение с NLP примери

### 4.1 Обучение с учител (Supervised Learning)

Имаме **етикетирани данни** и искаме да научим модел да предсказва етикети за нови данни.

#### Пример: Анализ на настроения (Sentiment Analysis)

**Задача**: Определи дали ревюто на филм е положително или отрицателно.

In [ ]:
# Създаване на малък dataset за анализ на настроения
# В реалната практика бихме използвали IMDB dataset с 50,000 ревюта

texts = [
    "Този филм е страхотен! Много ми хареса.",
    "Най-добрият филм, който съм гледал!",
    "Перфектно! Препоръчвам на всички.",
    "Чудесна история и отлични актьори.",
    "Прекрасен филм с много емоции.",
    "Ужасен филм. Пълна загуба на време.",
    "Най-лошият филм някога. Избягвайте го.",
    "Скучен и безинтересен. Не го препоръчвам.",
    "Разочароващ. Очаквах много повече.",
    "Лош сценарий и слаби актьори.",
    "Интересен филм с добра режисура.",
    "Харесах актьорската игра.",
    "Страхотни спецефекти и екшън.",
    "Впечатляващ филм! Заслужава гледане.",
    "Не е лош, но и не е страхотен."
]

# Етикети: 1 = положително, 0 = отрицателно
labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]

print("📊 Dataset за анализ на настроения")
print(f"Общо примери: {len(texts)}")
print(f"Положителни: {sum(labels)} | Отрицателни: {len(labels) - sum(labels)}")
print("\nПримери:")
for i in range(3):
    sentiment = "😊 Положително" if labels[i] == 1 else "😞 Отрицателно"
    print(f"{sentiment}: '{texts[i]}'")

### Разделяне на тренировъчен и тестов набор

**Важно**: Винаги разделяме данните преди да обучим модел!

In [ ]:
# Разделяне: 80% тренировъчен, 20% тестов
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print("🔄 Разделяне на данните:")
print(f"Тренировъчен набор: {len(X_train)} примера")
print(f"Тестов набор: {len(X_test)} примера")
print(f"\n✓ Използваме stratify за да запазим баланса на класовете в двата набора.")

### Извличане на характеристики и обучение на модел

In [ ]:
# Стъпка 1: Преобразуване на текст в числа (TF-IDF)
vectorizer = TfidfVectorizer(max_features=100)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("📐 Извличане на характеристики:")
print(f"Форма на тренировъчните данни: {X_train_vec.shape}")
print(f"  ({X_train_vec.shape[0]} документа, {X_train_vec.shape[1]} характеристики)")

# Стъпка 2: Обучение на класификатор (Logistic Regression)
model = LogisticRegression(random_state=42, max_iter=200)
model.fit(X_train_vec, y_train)

print("\n✓ Моделът е обучен!")

# Стъпка 3: Предсказване
y_pred = model.predict(X_test_vec)

# Стъпка 4: Оценка
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🎯 Точност на тестовия набор: {accuracy:.1%}")

### Нека да тестваме модела с нови текстове!

In [ ]:
# Нови примери за тестване
new_reviews = [
    "Великолепен филм! Ще го гледам отново.",
    "Ужасно. Съжалявам че похарчих пари за билет.",
    "Добър филм, но можеше да бъде по-добър."
]

print("🔮 Предсказване на настроението на нови ревюта:\n")

for review in new_reviews:
    # Преобразуване на текста
    review_vec = vectorizer.transform([review])
    
    # Предсказване
    prediction = model.predict(review_vec)[0]
    probability = model.predict_proba(review_vec)[0]
    
    sentiment = "😊 Положително" if prediction == 1 else "😞 Отрицателно"
    confidence = max(probability) * 100
    
    print(f"Ревю: '{review}'")
    print(f"  → Предсказание: {sentiment} (Увереност: {confidence:.1f}%)\n")

### Анализ: Кои думи са най-важни?

Нека да видим кои думи моделът счита за най-показателни за положителни и отрицателни настроения.

In [ ]:
# Получаване на коефициентите на модела
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

# Сортиране
top_positive_idx = np.argsort(coefficients)[-10:]
top_negative_idx = np.argsort(coefficients)[:10]

print("📈 Топ 10 думи за ПОЛОЖИТЕЛНО настроение:")
for idx in reversed(top_positive_idx):
    print(f"  • {feature_names[idx]}: {coefficients[idx]:.3f}")

print("\n📉 Топ 10 думи за ОТРИЦАТЕЛНО настроение:")
for idx in top_negative_idx:
    print(f"  • {feature_names[idx]}: {coefficients[idx]:.3f}")

print("\n💡 Положителните коефициенти показват думи, свързани с положително настроение.")
print("   Отрицателните коефициенти показват думи, свързани с отрицателно настроение.")

---
## 4.2 Обучение без учител (Unsupervised Learning)

Нямаме етикети. Целта е да открием **скрити структури** в данните.

### Пример: Групиране на документи (Document Clustering)

Представете си, че имаме новинарски статии и искаме да ги групираме по теми, без да знаем темите предварително.

In [ ]:
# Колекция от документи на различни теми
news_articles = [
    # Спорт
    "Левски победи ЦСКА с 2:0 в дербито на София.",
    "Григор Димитров достигна до финала на турнира в Лондон.",
    "Българският национален отбор спечели мача срещу Швейцария.",
    "Роналдо вкара два гола във вчерашния мач.",
    
    # Технологии
    "Apple представи новия iPhone с подобрена камера и батерия.",
    "Изкуственият интелект революционизира здравеопазването.",
    "Tesla пусна нова версия на автопилота за своите автомобили.",
    "Google разработва нов квантов компютър.",
    
    # Политика
    "Премиерът обяви нови мерки за икономическо развитие.",
    "Парламентът прие закон за реформа в образованието.",
    "Президентът участва в международна среща в Брюксел.",
    "Новият бюджет беше одобрен след дълги дебати."
]

print("📰 Новинарски статии за групиране:")
for i, article in enumerate(news_articles, 1):
    print(f"{i}. {article}")

print(f"\n💡 Имаме {len(news_articles)} статии. Можем ли да ги групираме по теми?")

In [ ]:
# Стъпка 1: Преобразуване в TF-IDF вектори
vectorizer_cluster = TfidfVectorizer(max_features=50)
article_vectors = vectorizer_cluster.fit_transform(news_articles)

print("📊 Преобразуване на текст в числа:")
print(f"Форма на матрицата: {article_vectors.shape}")

# Стъпка 2: Прилагане на K-Means клъстериране
# Казваме на алгоритъма да намери 3 групи (защото знаем, че има 3 теми)
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(article_vectors)

print(f"\n✓ Клъстерирането е завършено! Открити {n_clusters} групи.")

# Показваме резултатите
print("\n" + "="*70)
print("📑 РЕЗУЛТАТИ ОТ ГРУПИРАНЕТО")
print("="*70)

for cluster_id in range(n_clusters):
    print(f"\n🏷️  ГРУПА {cluster_id}:")
    indices = np.where(clusters == cluster_id)[0]
    for idx in indices:
        print(f"  • {news_articles[idx]}")

### Интерпретация

**Забележете**: Алгоритъмът **сам** откри че:
- Група 0: Спортни новини
- Група 1: Технологични новини
- Група 2: Политически новини

Никой не му каза кои са темите - той ги откри от сходството в думите!

**Реални приложения**:
- Организиране на документи в библиотеки
- Групиране на клиентски оплаквания по теми
- Откриване на теми в социални медии

---
## 4.3 Обучение чрез подкрепление (Reinforcement Learning)

Агентът учи чрез **опит и грешка**, получавайки награди или наказания.

### Връзка с NLP

**RLHF (Reinforcement Learning from Human Feedback)** е ключова техника за обучение на съвременни LLM:

1. Моделът генерира отговор
2. Човек оценява качеството (награда)
3. Моделът учи да генерира по-добри отговори

**Примери**:
- ChatGPT и Claude са обучени с RLHF
- Диалогови системи, които учат от потребителски взаимодействия
- Системи за машинен превод с човешка обратна връзка

📌 **Ще разгледаме RLHF подробно в Лекция 8!**

---
## 5. Генерализация: Overfitting и Underfitting

### Какво е генерализация?

**Генерализацията** е способността на модел да работи добре на **нови, невиждани данни**.

### Две основни проблеми

#### 🔴 Overfitting (Препълване)
Моделът **запомня** тренировъчните данни, но не може да генерализира.

**В текстови модели**:
- Запомня точни изречения от тренировъчния набор
- Не работи добре с нови формулировки
- Не разпознава невиждани думи

#### 🔵 Underfitting (Недопълване)
Моделът е **твърде прост** и не може да улови модели в данните.

**В текстови модели**:
- Bag-of-words, който пропуска реда на думите
- Прекалено прости характеристики
- Игнориране на контекст

### Демонстрация с текстови данни

In [ ]:
# Създаване на малък dataset за демонстрация на overfitting
small_texts = [
    "Страхотен филм",
    "Много добър филм",
    "Лош филм",
    "Ужасен филм"
]
small_labels = [1, 1, 0, 0]  # 1=положително, 0=отрицателно

# Ще тренираме модели с различна сложност
print("🔬 Експеримент: Влияние на сложността на модела\n")

# Модел 1: Прост (само 5 характеристики)
vec_simple = TfidfVectorizer(max_features=5)
X_simple = vec_simple.fit_transform(small_texts)
model_simple = LogisticRegression(random_state=42, C=0.1)
model_simple.fit(X_simple, small_labels)
train_acc_simple = model_simple.score(X_simple, small_labels)

print(f"📊 Прост модел (5 характеристики):")
print(f"   Точност на тренировъчните данни: {train_acc_simple:.1%}")
print(f"   Речник: {vec_simple.get_feature_names_out()}")

# Модел 2: Сложен (всички думи + n-грами)
vec_complex = TfidfVectorizer(ngram_range=(1, 3), max_features=100)
X_complex = vec_complex.fit_transform(small_texts)
model_complex = LogisticRegression(random_state=42, C=100)
model_complex.fit(X_complex, small_labels)
train_acc_complex = model_complex.score(X_complex, small_labels)

print(f"\n📊 Сложен модел ({X_complex.shape[1]} характеристики):")
print(f"   Точност на тренировъчните данни: {train_acc_complex:.1%}")
print(f"   Използва 1-грами, 2-грами и 3-грами")

# Тестване на нови фрази
test_phrases = [
    "Прекрасен филм",  # Положително, но нова дума
    "Отвратителен филм",  # Отрицателно, но нова дума
]

print("\n" + "="*70)
print("🧪 Тестване на нови фрази (думи, които не са в тренировъчния набор):")
print("="*70)

for phrase in test_phrases:
    # Прост модел
    phrase_simple = vec_simple.transform([phrase])
    pred_simple = model_simple.predict(phrase_simple)[0]
    
    # Сложен модел
    phrase_complex = vec_complex.transform([phrase])
    pred_complex = model_complex.predict(phrase_complex)[0]
    
    print(f"\nФраза: '{phrase}'")
    print(f"  Прост модел: {'😊 Положително' if pred_simple == 1 else '😞 Отрицателно'}")
    print(f"  Сложен модел: {'😊 Положително' if pred_complex == 1 else '😞 Отрицателно'}")

print("\n💡 Забележка: На малки данни, сложният модел може да overfitне,")
print("   докато простият модел може да underfitне.")
print("   Балансът е ключов!")

### Как да избегнем overfitting?

1. **Повече тренировъчни данни** 📊
2. **Регуларизация** (L1, L2) ⚖️
3. **Кръстосана валидация** 🔄
4. **По-прости модели** 📉
5. **Early stopping** ⏸️

### Връзка с LLM

Съвременните LLM (ChatGPT, Claude) се обучават на **трилиони** токени, което им позволява да генерализират невероятно добре!

📌 **Ще разгледаме scaling laws в Лекция 6-7!**

---
## 6. Метрики за оценка

### За класификация на текст

#### Точност (Accuracy)
$$\text{Accuracy} = \frac{\text{Брой правилни предсказания}}{\text{Общ брой предсказания}}$$

#### Прецизност (Precision)
От всички предсказани като положителни, колко наистина са положителни?
$$\text{Precision} = \frac{TP}{TP + FP}$$

#### Пълнота (Recall)
От всички действително положителни, колко сме открили?
$$\text{Recall} = \frac{TP}{TP + FN}$$

#### F1-Score
Хармонична средна на Precision и Recall
$$F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

In [ ]:
# Използваме предсказанията от по-рано
print("📊 ДЕТАЙЛНИ МЕТРИКИ ЗА КЛАСИФИКАЦИЯ НА НАСТРОЕНИЯ\n")

# Classification report
print("Детайлен доклад:")
print("=" * 60)
report = classification_report(y_test, y_pred, 
                               target_names=['Отрицателно', 'Положително'],
                               zero_division=0)
print(report)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Отрицателно', 'Положително'],
            yticklabels=['Отрицателно', 'Положително'],
            cbar_kws={'label': 'Брой предсказания'})
plt.ylabel('Истинска стойност', fontsize=12)
plt.xlabel('Предсказана стойност', fontsize=12)
plt.title('Матрица на объркване (Confusion Matrix)', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("\n📖 Как да четем матрицата:")
print(f"  ✓ Горе-ляво: Правилно предсказани отрицателни ({cm[0,0]})")
print(f"  ✓ Долу-дясно: Правилно предсказани положителни ({cm[1,1]})")
print(f"  ✗ Горе-дясно: Фалшиви положителни - грешка тип I ({cm[0,1]})")
print(f"  ✗ Долу-ляво: Фалшиви отрицателни - грешка тип II ({cm[1,0]})")

### Специални NLP метрики

#### Perplexity (за езикови модели)
Измерва колко "объркан" е моделът при предсказване на следващата дума.

$$\text{Perplexity} = 2^{-\frac{1}{N}\sum_{i=1}^{N}\log_2 P(w_i|w_1,...,w_{i-1})}$$

По-ниска perplexity = по-добър модел

#### BLEU Score (за машинен превод)
Сравнява генериран превод с референтни преводи, базирайки се на съвпадение на n-грами.

📌 **Ще видим тези метрики в следващите лекции!**

---
## 7. NLP-специфични предизвикателства

### 1. Представяне на текст като числа
- Bag of Words ✓ (видяхме)
- TF-IDF ✓ (видяхме)
- Word2Vec → **Лекция 2**
- Embeddings → **Лекция 2**
- Contextualized embeddings (BERT) → **Лекция 5**

### 2. Токенизация
Как разделяме текст на части?
- Думи
- Поддуми (BPE, WordPiece)
- Символи

📌 **Цяла лекция за това → Лекция 3**

### 3. Променлива дължина на последователностите
Изреченията имат различна дължина. Как ги обработваме?
- Padding (запълване)
- Truncation (отрязване)
- Рекурентни мрежи (RNN)
- Attention механизми

📌 **Трансформери → Лекция 5**

### 4. Контекст и значение
"Банката е затворена" - финансова институция или речен бряг?

Традиционни методи (Bag of Words) игнорират контекст.  
Съвременни модели (BERT, GPT) използват контекст!

### 5. Мащаб на данните
Съвременните LLM се обучават на:
- Трилиони токени
- Стотици GB текст
- Хиляди GPU часове

📌 **Scaling laws → Лекция 6-7**

---
## 8. Практически NLP workflow

### Стъпки при решаване на NLP задача

```
1. Дефиниране на задачата
   ↓
2. Събиране и проучване на текстови данни
   ↓
3. Предобработка (lowercase, премахване на пунктуация, ...)
   ↓
4. Токенизация
   ↓
5. Извличане на характеристики / Embeddings
   ↓
6. Разделяне на train/val/test
   ↓
7. Избор и обучение на модел
   ↓
8. Оценка на тестовия набор
   ↓
9. Анализ на грешки
   ↓
10. Итерация и подобрение
```

### Често срещани капани в NLP

⚠️ **Data leakage**: Информация от тестовия набор "изтича" в тренировъчния  
⚠️ **Test set contamination**: Особено проблем за LLM (виждали ли са данните?)  
⚠️ **Vocabulary mismatch**: Модел вижда нови думи на тест време  
⚠️ **Class imbalance**: Неравномерно разпределение на класовете  
⚠️ **Domain shift**: Тренирани на новини, тествани на туитове

In [ ]:
# Демонстрация на vocabulary mismatch проблем
print("⚠️  ДЕМОНСТРАЦИЯ: Vocabulary Mismatch Problem\n")

train_texts_demo = ["добър филм", "лош филм", "страхотен филм"]
train_labels_demo = [1, 0, 1]

# Обучаваме модел
vec_demo = CountVectorizer()
X_train_demo = vec_demo.fit_transform(train_texts_demo)
model_demo = LogisticRegression()
model_demo.fit(X_train_demo, train_labels_demo)

print("Тренировъчен речник:")
print(vec_demo.get_feature_names_out())

# Тестваме с текст, съдържащ нови думи
test_text = ["превъзходен филм"]
print(f"\nТестов текст: '{test_text[0]}'")
print("Проблем: Думата 'превъзходен' не е във речника!")

X_test_demo = vec_demo.transform(test_text)
print(f"\nПредставяне на тестовия текст: {X_test_demo.toarray()}")
print("→ Всички характеристики са 0! Моделът 'не вижда' новата дума.")

print("\n💡 Решения:")
print("  1. Използвай embeddings вместо bag-of-words (Лекция 2)")
print("  2. Обучавай на по-големи и разнообразни datasets")
print("  3. Използвай поддумен токенизация (BPE) (Лекция 3)")

---
## 9. Обобщение и мост към следващата лекция

### 🎓 Ключови изводи

#### Днес научихме:

1. **ML основи приложени за NLP**
   - Формулиране на задачи с текстови данни
   - Характеристики: Bag of Words, TF-IDF, n-грами

2. **Три парадигми на обучение**
   - **Supervised**: Анализ на настроения, класификация на текст
   - **Unsupervised**: Групиране на документи, откриване на теми
   - **Reinforcement**: RLHF за обучение на LLM (ще видим в Лекция 8)

3. **Важни концепции**
   - Train/Test split за валидна оценка
   - Overfitting и underfitting в текстови модели
   - Метрики: Accuracy, Precision, Recall, F1

4. **NLP предизвикателства**
   - Представяне на текст като числа
   - Променлива дължина на последователности
   - Важността на контекст
   - Vocabulary mismatch

### 🔜 Следваща лекция: Езикови модели и представяне на думи

**Ще отговорим на критични въпроси:**

❓ Как представяме думите по-добре от Bag of Words?  
❓ Как улавяме семантична прилика? ("крал" ≈ "кралица")  
❓ Какво е Word2Vec и как работи?  
❓ Какво е езиков модел?  
❓ Как измерваме качеството му (Perplexity)?  

### 📚 Препоръчително четене

1. **"A Few Useful Things to Know About Machine Learning"** - Pedro Domingos (2012)
2. **"Speech and Language Processing"** (3rd ed.) - Jurafsky & Martin
   - Chapter 4: Naive Bayes and Sentiment Classification
   - Chapter 6: Vector Semantics and Embeddings (преглед за следващата лекция)
3. **Stanford CS224N Lecture 1**: Introduction to NLP and Deep Learning

### 🏋️ Упражнения за вкъщи

1. Изтеглете IMDB dataset и обучете sentiment classifier
2. Сравнете Bag-of-Words vs TF-IDF vs n-грами
3. Анализирайте грешките на модела - кои типове изречения са трудни?
4. Експериментирайте с различни стойности на max_features

---
## Край на Лекция 1

### Благодаря за вниманието! 🎉

**Въпроси? 🤔**